# MAIN WORKING BODY

#### Import stuff and creating drivers

In [121]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

import pandas as pd

driver = webdriver.Chrome('/home/jaspreet/Documents/chromedriver')

#### Initialising the Driver with Indeed.co

In [122]:
driver.get('https://www.indeed.co.in/')
driver.refresh()

#### Enter Jobs and location for the first time 

In [46]:
jobs = input('Enter job: ')
loc = input('Enter location: ')

Enter job: Sales
Enter location: Delhi


#### Input the Job and Location into the text fields and search 

In [123]:
what = driver.find_element_by_id('text-input-what')
where = driver.find_element_by_id('text-input-where')

what.send_keys(Keys.CONTROL + "a")
what.send_keys(Keys.DELETE)
what.send_keys(jobs)

where.send_keys(Keys.CONTROL + "a")
where.send_keys(Keys.DELETE)
where.send_keys(loc)
where.send_keys(Keys.RETURN)

#### Place holding DF till I figure out something better.

In [115]:
df = pd.DataFrame(columns=('Job_Title', 'Company_Name', 'Location', 'Salary', 'Job_Summary'))

#### Iterate over the first nine pages

In [128]:
# Till we don't reach the 10th page
count = 1
while '3' not in driver.find_element_by_xpath("//div[@class = 'pagination']/b").text : 
    print('Page :', count)
    count = count + 1

    # Get source
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')  
    
    # Make a SOUP iterable list
    SOUP_JOBS = soup.find_all(class_= 'jobsearch-SerpJobCard unifiedRow row result clickcard')
    # Make a SELENIUM iterable list
    SELENIUM_JOBS = driver.find_elements_by_xpath("//div[@class = 'location accessible-contrast-color-location']")
    
    
    # Max_iterator = len(SOUP_JOBS)
    iterator = 0
    
    # Run loop to store things in CSV
    for i in range(5) :
        
        
        # Make a SOUP iterable list
        SOUP_JOBS = soup.find_all(class_= 'jobsearch-SerpJobCard unifiedRow row result clickcard')
        # Make a SELENIUM iterable list
        SELENIUM_JOBS = driver.find_elements_by_xpath("//div[@class = 'location accessible-contrast-color-location']")
        # False consolation to ego, LOL
        print('Done with a Job!')
        
        # Make variables to store info and use later
        Job_Title = SOUP_JOBS[i].find(attrs={'data-tn-element' : 'jobTitle'})['title']
        Company_Name = SOUP_JOBS[i].find(class_ = 'company').text
        Location = SOUP_JOBS[i].find(class_ = 'location accessible-contrast-color-location').text
        Salary = 'NEGOTIABLE'
        if SOUP_JOBS[i].find(class_ = 'salaryText') != None:
            Salary = SOUP_JOBS[i].find(class_ = 'salaryText').text
        Job_Summary = ''
        
        
        '''Tackle Summary waala problem'''
        
        # Find element number
        index_num = iterator
        print('Found job: ', index_num)
        print('New Index:', index_num)
        
        # Focus on element
        summary_page = SELENIUM_JOBS[index_num]
        print('Accessing job element...')
        print(summary_page)
        # Click on element
        summary_page.click()
        print('Clicked on job.')
        
        # Only if new page has opened
        num_tabs = driver.window_handles
        if len(num_tabs) == 2:
            
            print('Entered new tab.')
            # Switch to new tab
            driver.switch_to_window(num_tabs[1])
            
            # Find and store Job_Summary
            Job_Summary = driver.find_element_by_xpath("//div[@class='jobsearch-ViewJobLayout-jobDisplay icl-Grid-col icl-u-xs-span12 icl-u-lg-span7']")
            Job_Summary = Job_Summary.text
            
            # Move on in life
            driver.close()
            print('Closed tab.')
            driver.switch_to_window(num_tabs[0])
            print('Switched back')
            
            if len(driver.window_handles) != 1:
                print('Mil gaya')
                    
        else:
            print('OHHHHH!!! OHHHHHHHHHH!!! ERROR!!!!!!')
            print('Please, PLEASE, call Jaspreet!')
            break
        
        
        
        #creating an entry to add to CSV file
        csv_dict = [{'Job_Title':Job_Title, 'Company_Name':Company_Name, 'Location':Location, 'Salary':Salary, 'Job_Summary':Job_Summary}]
        temp_df_entry = pd.DataFrame(csv_dict)
        
        #pushing entry into CSV file
        df = df.append(temp_df_entry, ignore_index = True)
        df.to_csv('God_Given_Gift.csv')
        
        # Click the next page
        next_page = driver.find_element_by_class_name('np')
        next_page.click()
        iterator = iterator + 1

Page : 1
Done with a Job!
Found job:  0
New Index: 0
Accessing job element...
<selenium.webdriver.remote.webelement.WebElement (session="d03a027a2a289f3cc738597bdd355321", element="f37a2b9b-2619-40b2-9ce7-0744ec8a455a")>
Clicked on job.
Entered new tab.


/home/jaspreet/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: DeprecationWarning: use driver.switch_to.window instead


Closed tab.
Switched back


/home/jaspreet/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:71: DeprecationWarning: use driver.switch_to.window instead


Done with a Job!
Found job:  1
New Index: 1
Accessing job element...
<selenium.webdriver.remote.webelement.WebElement (session="d03a027a2a289f3cc738597bdd355321", element="ff2c520e-00d8-445e-bdbb-1b2f2e5fa667")>
Clicked on job.
Entered new tab.
Closed tab.
Switched back
Done with a Job!
Found job:  2
New Index: 2
Accessing job element...
<selenium.webdriver.remote.webelement.WebElement (session="d03a027a2a289f3cc738597bdd355321", element="29bf31cd-591e-4a37-b183-08052880989c")>
Clicked on job.
Entered new tab.
Closed tab.
Switched back
Done with a Job!
Found job:  3
New Index: 3
Accessing job element...
<selenium.webdriver.remote.webelement.WebElement (session="d03a027a2a289f3cc738597bdd355321", element="6bf6af4a-e663-4e7c-a0bf-18b430c61f5e")>
Clicked on job.
Entered new tab.
Closed tab.
Switched back
Done with a Job!
Found job:  4
New Index: 4
Accessing job element...
<selenium.webdriver.remote.webelement.WebElement (session="d03a027a2a289f3cc738597bdd355321", element="24e56ffc-3e36-4

KeyboardInterrupt: 

In [87]:
for i in SOUP_JOBS:
    print(i)
    print('******************')
    print()

<div class="jobsearch-SerpJobCard unifiedRow row result clickcard" data-ci="317194622" data-empn="964103211183813" data-jk="d407285327f54a69" id="pj_d407285327f54a69">
<div class="title">
<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0BWSdDu9OymXJqz7vnF3B8_euqfxqxvesAEeS3x-pQdyjivtEMoAtaFUN-9y5sgUPFYKHMn8w149G5UkhdX5okr2YilzabE8XC1r3zGSlAFT3djGaazmndUmFakv4blToIMLAz92-7QIPVGioEBv1Di_zXw940Hz06qC6OC_ybJYVonfYUdqEhLmfA6x6WruCSzCZHiqMuAMCMnJjKgNMEilImLWYvRZ5Y9CADGGwtlAmAgIH4zBdBq6OyGMiSwSud_wOthug8lqYyMdnMgGSUmMvraiYPoFvZnNT1EMjhpiQ4YbTk6GcO4SoGc6qoDgCd7ml9vmS5EWBt9RchJviz0qdIpAixkQZzPkOnpr8w8C0s7bRbgDfBDHAEPZ7PDyxxCPcyCpgp_O8Y7GPJWvmusBJbpLtmKfZw5OJpBiSxQEA==&amp;p=0&amp;fvj=1&amp;vjs=3" id="sja0" onclick="setRefineByCookie([]); sjoc('sja0', 1); convCtr('SJ'); rclk(this,jobmap[0],true,1);" onmousedown="sjomd('sja0'); clk('sja0'); rclk(this,jobmap[0],1);" rel="noopener nofollow" target="_blank" title="Direct Sales Associate">
Direct <b

In [101]:
def highlight(element, effect_time, color, border):
    """Highlights (blinks) a Selenium Webdriver element"""
    driver = element._parent
    def apply_style(s):
        driver.execute_script("arguments[0].setAttribute('style', arguments[1]);",
                              element, s)
    original_style = element.get_attribute('style')
    apply_style("border: {0}px solid {1};".format(border, color))
    time.sleep(effect_time)
    apply_style(original_style)

In [107]:
j = 1
for i in SELENIUM_JOBS:
    highlight(i, 3, "blue", 5)
    print('******************')
    print()

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=78.0.3904.97)
